# Métodos Alternativos de Rastreamento

![AWTT](../../images/alternative_ways_to_trace_0.png)

Até agora neste módulo, analisamos o decorador traceable e como podemos usá-lo para configurar o rastreamento.

Nesta lição, vamos analisar maneiras alternativas de configurar o rastreamento e quando você deve pensar em usar essas diferentes abordagens.

## LangChain e LangGraph

Se estivermos usando LangChain ou LangGraph, tudo o que precisamos fazer para configurar o rastreamento é definir algumas variáveis de ambiente

![AWTT](../../images/alternative_ways_to_trace_1.png)

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # Se você não definir isso, os rastreamentos irão para o projeto Default

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

Não se preocupe muito com nossa implementação de grafo aqui, você pode aprender mais sobre LangGraph através do nosso curso LangGraph Academy!

In [ ]:
import nest_asyncio
import operator
from langchain.schema import Document
from langchain_core.messages import HumanMessage, AnyMessage, get_buffer_string
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from typing import List
from typing_extensions import TypedDict, Annotated
from utils import get_vector_db_retriever, RAG_PROMPT

nest_asyncio.apply()

retriever = get_vector_db_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# Define Graph state
class GraphState(TypedDict):
    question: str
    messages: Annotated[List[AnyMessage], operator.add]
    documents: List[Document]

# Define Nodes
def retrieve_documents(state: GraphState):
    messages = state.get("messages", [])
    question = state["question"]
    documents = retriever.invoke(f"{get_buffer_string(messages)} {question}")
    return {"documents": documents}

def generate_response(state: GraphState):
    question = state["question"]
    messages = state["messages"]
    documents = state["documents"]
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
    rag_prompt_formatted = RAG_PROMPT.format(context=formatted_docs, conversation=messages, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    return {"documents": documents, "messages": [HumanMessage(question), generation]}

# Define Graph
graph_builder = StateGraph(GraphState)
graph_builder.add_node("retrieve_documents", retrieve_documents)
graph_builder.add_node("generate_response", generate_response)
graph_builder.add_edge(START, "retrieve_documents")
graph_builder.add_edge("retrieve_documents", "generate_response")
graph_builder.add_edge("generate_response", END)

simple_rag_graph = graph_builder.compile()
display(Image(simple_rag_graph.get_graph().draw_mermaid_png()))

Estamos configurando um grafo simples no LangGraph. Se você quiser aprender mais sobre LangGraph, eu recomendaria fortemente dar uma olhada em nosso curso LangGraph Academy.

Você também pode passar metadados ou outros campos através de uma configuração opcional

In [ ]:
question = "Como configurar rastreamento se estou usando LangChain?"
simple_rag_graph.invoke({"question": question}, config={"metadata": {"foo": "bar"}})

##### Vamos dar uma olhada no LangSmith!

## Gerenciador de Contexto de Rastreamento

Em Python, você pode usar o gerenciador de contexto trace para registrar rastreamentos no LangSmith. Isso é útil em situações onde:

Você quer registrar rastreamentos para um bloco específico de código.
Você quer controle sobre as entradas, saídas e outros atributos do rastreamento.
Não é viável usar um decorador ou wrapper.
Qualquer um ou todos os itens acima.
O gerenciador de contexto se integra perfeitamente com o decorador traceable e o wrapper wrap_openai, então você pode usá-los juntos na mesma aplicação.

Você ainda precisa definir seu `LANGSMITH_API_KEY` e `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_2.png)

In [ ]:
from langsmith import traceable, trace
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat

MODEL_PROVIDER = "google"
MODEL_NAME = "gemini-1.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Retorna documentos buscados de um armazenamento vetorial baseado na pergunta do usuário
"""
@traceable
def retrieve_documents(question: str):
    documents = retriever.invoke(question)
    return documents

"""
generate_response
- Chama `call_gemini` para gerar uma resposta do modelo após formatar entradas
"""
# TODO: Remove traceable, and use with trace()
@traceable
def generate_response(question: str, documents):
    # NOTA: Nossos documentos vieram como uma lista de objetos, mas queremos apenas registrar uma string
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    # TODO: Use with trace()
    # with trace(
    #     name="Gerar Resposta",
    #     run_type="chain",
    #     inputs={"question": question, "formatted_docs": formatted_docs},
    #     metadata={"foo": "bar"},
    # ) as ls_trace:
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    response = call_gemini(messages)
    # TODO: Finalize seu rastreamento e escreva as saídas para o LangSmith
    # ls_trace.end(outputs={"output": response})
    return response

"""
call_gemini
- Retorna a saída de conclusão de chat do Google Gemini
"""
@traceable(
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_gemini(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return call_gemini_chat(model, messages, temperature)

"""
langsmith_rag
- Chama `retrieve_documents` para buscar documentos
- Chama `generate_response` para gerar uma resposta baseada nos documentos buscados
- Retorna a resposta do modelo
"""
@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

In [ ]:
question = "Como rastreio com contexto de rastreamento?"
ai_answer = langsmith_rag(question)
print(ai_answer)

## Integrações Gemini com LangSmith

Para modelos Google Gemini, utilizamos funções utilitárias personalizadas para integração com LangSmith, já que não há wrapper oficial equivalente ao wrap_openai. O decorador @traceable funciona perfeitamente com chamadas Gemini através de nossas funções helper.

Você ainda precisa definir seu `GOOGLE_API_KEY` e `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_3.png)

In [ ]:
from langsmith import traceable
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat

MODEL_PROVIDER = "google"
MODEL_NAME = "gemini-1.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_gemini(messages)

@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_gemini(
    messages: List[dict],
) -> str:
    return call_gemini_chat(MODEL_NAME, messages, 0.0)

@traceable(run_type="chain")
def langsmith_rag_with_gemini(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

In [ ]:
question = "Como rastreio com Google Gemini?"
ai_answer = langsmith_rag_with_gemini(question)
print(ai_answer)

As funções Gemini com @traceable aceitas parâmetros langsmith_extra similares às funções decoradas

In [ ]:
question = "Qual é a cor do céu?"
ai_answer = langsmith_rag_with_gemini(
    question, 
    langsmith_extra={"metadata": {"foo": "bar"}}
)
print(ai_answer)

## [Avançado] RunTree

Outra maneira mais explícita de registrar rastreamentos no LangSmith é através da API RunTree. Esta API permite mais controle sobre seu rastreamento - você pode criar manualmente execuções e execuções filhas para montar seu rastreamento. Você ainda precisa definir seu `LANGSMITH_API_KEY`, mas `LANGSMITH_TRACING` não é necessário para este método.

![AWTT](../../images/alternative_ways_to_trace_4.png)

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
from dotenv import load_dotenv
# Tenho minhas variáveis de ambiente definidas em um arquivo .env
load_dotenv(dotenv_path="../../.env", override=True)

Vamos definir `LANGSMITH_TRACING` como false, pois estamos usando RunTree para criar execuções manualmente neste caso.

In [ ]:
import os
os.environ["LANGSMITH_TRACING"] = "false"

from langsmith import utils
utils.tracing_is_enabled() # Isso deve retornar false

Reescrevemos nossa aplicação RAG, exceto que desta vez passamos um argumento RunTree através de nossas chamadas de função e criamos execuções filhas em cada camada. Isso dá ao nosso RunTree a mesma hierarquia que pudemos estabelecer automaticamente com @traceable

In [ ]:
from langsmith import RunTree
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat

nest_asyncio.apply()
retriever = get_vector_db_retriever()

def retrieve_documents(parent_run: RunTree, question: str):
    # Criar uma execução filha
    child_run = parent_run.create_child(
        name="Recuperar Documentos",
        run_type="retriever",
        inputs={"question": question},
    )
    documents = retriever.invoke(question)
    # Postar a saída de nossa execução filha
    child_run.end(outputs={"documents": documents})
    child_run.post()
    return documents

def generate_response(parent_run: RunTree, question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """Você é um assistente para tarefas de perguntas e respostas.
    Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
    Se você não souber a resposta, apenas diga que não sabe.
    Use no máximo três frases e mantenha a resposta concisa.
    """
    # Criar uma execução filha
    child_run = parent_run.create_child(
        name="Gerar Resposta",
        run_type="chain",
        inputs={"question": question, "documents": documents},
    )
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    gemini_response = call_gemini(child_run, messages)
    # Postar a saída de nossa execução filha
    child_run.end(outputs={"gemini_response": gemini_response})
    child_run.post()
    return gemini_response

def call_gemini(
    parent_run: RunTree, messages: List[dict], model: str = "gemini-1.5-flash", temperature: float = 0.0
) -> str:
    # Criar uma execução filha
    child_run = parent_run.create_child(
        name="Chamada Gemini",
        run_type="llm",
        inputs={"messages": messages},
        metadata={
            "ls_provider": "google",
            "ls_model_name": model
        }
    )
    gemini_response = call_gemini_chat(model, messages, temperature)
    # Postar a saída de nossa execução filha
    child_run.end(outputs={"gemini_response": gemini_response})
    child_run.post()
    return gemini_response

def langsmith_rag(question: str):
    # Criar um RunTree raiz
    root_run_tree = RunTree(
        name="Pipeline de Chat",
        run_type="chain",
        inputs={"question": question}
    )

    # Passar nosso RunTree para as chamadas de função aninhadas
    documents = retrieve_documents(root_run_tree, question)
    response = generate_response(root_run_tree, question, documents)
    output = response

    # Postar nossa saída final
    root_run_tree.end(outputs={"generation": output})
    root_run_tree.post()
    return output
    

In [ ]:
question = "Como posso rastrear com RunTree?"
ai_answer = langsmith_rag(question)
print(ai_answer)